In [ ]:
# ===========================================================================
# Testing the postprocessing pipeline analyses for the sorted Wifi_20240430 session with kilosort4
# ===========================================================================
import spikeinterface as si


from pathlib import Path

# Loading the bin data 
file_path = Path(r"C:\Users\boninilab\Desktop\kilosort-test\sorting-test-Wifi_20240430\binary_files\Wifi_20240430_good_heatmap.bin")

# Confirm file existence
assert file_path.is_file(), f"Error: {file_path} is not a valid file. Please check the path."

# Define recording parameters
sampling_frequency = 32000.0  
num_channels = 128  
dtype = "uint16"  

# Load data using SpikeInterface
recording = si.read_binary(
    file_paths=file_path,
    sampling_frequency=sampling_frequency,
    num_channels=num_channels,
    dtype=dtype
)

# Load the probe configuration
from probeinterface import read_prb

probe_file = Path("wifi_channel_map_kilosort.prb")
probe = read_prb(probe_file)

recording.set_probegroup(probe, in_place=True)

recording

BinaryRecordingExtractor: 128 channels - 32.0kHz - 1 segments - 128,791,427 samples 
                          4,024.73s (1.12 hours) - uint16 dtype - 30.71 GiB
  file_paths: ['C:\\Users\\boninilab\\Desktop\\kilosort-test\\sorting-test-Wifi_20240430\\binary_files\\Wifi_20240430_good_heatmap.bin']

In [2]:
# Load the sorting result from kilosort4/phy
from spikeinterface.extractors import read_phy

sorting_path = Path(r"C:\Users\boninilab\Desktop\kilosort-test\sorting-test-Wifi_20240430\kilosort4_output_tests\Wifi_20240430_test3")
sorting = read_phy(sorting_path)
sorting

PhySortingExtractor: 86 units - 1 segments - 32.0kHz

In [3]:
# Create the SortingAnalyzer

analyzer = si.create_sorting_analyzer(
    sorting=sorting,
    recording=recording,
    sparse=True, # default
    format="memory", # default
)
print(analyzer)

estimate_sparsity (no parallelization):   0%|          | 0/4025 [00:00<?, ?it/s]

SortingAnalyzer: 128 channels - 86 units - 1 segments - memory - sparse - has recording
Loaded 0 extensions


In [ ]:
analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
analyzer.compute("waveforms",  ms_before=1,ms_after=2.)
analyzer.compute("templates", operators=["average", "median", "std"])
analyzer.compute("noise_levels")
analyzer

compute_waveforms (no parallelization):   0%|          | 0/4025 [00:00<?, ?it/s]

ValueError: This recording does not support return_scaled=True (need gain_to_uV and offset_to_uV properties)